In [ ]:
#import libraries
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Question 1


Scrape 3 pages of reviews for a single product listing using product URL.<br /> Note: Product must have more than 1 review page. The code must be generic and should work for every product listing URL, with proper checks in place.

In [ ]:
url = 'https://www.daraz.pk/products/tecno-camon-19-neo-128gb-rom-6gb-ram-rear-camera-48mp-1-year-brand-warranty-with-mobile-phone-i336664418-s1743322836.html?spm=a2a0e.searchlistcategory.list.1.56cfc414HT8f4v&search=1'

options = Options()
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(url)
driver.maximize_window()
time.sleep(5)
users =[]
reviews = []
time.sleep(5)
for k in range(0,3):
    time.sleep(8)
    main = driver.find_element('xpath', '//*[@id="module_product_review"]')
    time.sleep(8)
    driver.execute_script("arguments[0].scrollIntoView(true)", main)
    time.sleep(5)
    revs = driver.find_elements('class name', "item")
    for i in revs:
        try:
            l = (i.text.split("\n"))
            users.append(l[1].split(".")[0].replace("by ", "").replace("Verified Purchase", ""))
            reviews.append(l[2])
        except:
            driver.quit()
    ele=driver.find_element('xpath','//*[@id="module_product_review"]/div/div/div[3]/div[2]/div/button[2]')
    time.sleep(5)
    ele.click()
    time.sleep(5)
driver.quit()


C:\Users\FB TRADERS\AppData\Local\Temp\ipykernel_12508\3589918568.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [ ]:
data={'Name':users,'Review text':reviews}
df=pd.DataFrame(data)
df

,Name,Review text
0,A***,Thanks daraz such a great deals on Azaadi Muba...
1,Faraz A,Original phone. Received well packed at a very...
2,Bilal H,بہت ہی کمال کا موبائل بھیجا ہے ہے میری زندگی س...
3,Taimour,Handfree missing in box but received as it is ...
4,3***2,bhot acha products hai par Handsfree nhi hai. ...
5,3***0,awsome jo mangwaya wo mila same day order same...
6,umair I,original product . received in good condition ...
7,Ayaz A,It's my 2nd great experience with same vendor ...
8,Butt S,i purchase the phone in night and delivery in ...
9,3***3,"amazing mobile good features , good battery ti..."


### Question 1 Part b

Clean the Reviews using NLTK,RE. Remove Punctuations, Smilies and other garbage using any library.

Apply Vader Sentiment to find the sentiment of the review, and append it in a column named "Sentiment".

You can get help of Vader sentiment from this; https://towardsdatascience.com/sentimental-analysis-using-vader-a3415fef7664

In [ ]:
def remove_stopwords(word_list):
        st = set(stopwords.words('english'))
        word=[]
        for i in word_list:
            if not i.lower() in st:
                word.append(i);
        return word
def remove_pun(st):             #remove punctuation form the string
    pun=[]
    for i in st:
        if(not i in punctuation):     #if no punctuation append in list
            pun.append(i)
    return pun

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
import string
punctuation=string.punctuation
score=[]
for i in range(0,len(reviews)):
    word=remove_pun(reviews[i])
    word=''.join(word)
    reviews[i]=word
    word=remove_stopwords(reviews[i].split(' '))
    word=' '.join(word)
    reviews[i]=word
    analysis = SentimentIntensityAnalyzer()
    if analysis.polarity_scores(reviews[i])['compound'] > 0:
        score.append('Positive')
    else:
        score.append('Negative')
df['Sentiment']=score
df['Review text']=reviews
df

,Name,Review text,Sentiment
0,A***,Thanks daraz great deals Azaadi Mubarik Great ...,Positive
1,Faraz A,Original phone Received well packed good price...,Positive
2,Bilal H,بہت ہی کمال کا موبائل بھیجا ہے ہے میری زندگی س...,Negative
3,Taimour,Handfree missing box received show ad,Negative
4,3***2,bhot acha products hai par Handsfree nhi hai b...,Negative
5,3***0,awsome jo mangwaya wo mila day order day deliv...,Negative
6,umair I,original product received good condition good...,Positive
7,Ayaz A,2nd great experience vendor Daraz 1st one Camo...,Positive
8,Butt S,purchase phone night delivery next morning fas...,Positive
9,3***3,amazing mobile good features good battery tim...,Positive


# Question 2

Scrape 80 items for a keyword search e.g., 'mobile covers'<br/>Note: Must use "click()" function of webElement, URL changing for pages is not allowed. The code must be generic and should work for every keyword, with proper checks in place. Bonus for also scraping country and Daraz Mall availability.

In [ ]:
q = input("Please enter search term: ").replace(' ', '+')
url = f'https://www.daraz.pk/catalog/?q={q}'
options = Options()
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(url)
driver.maximize_window()
names=[]
prices=[]
dmall=[]
reviews=[]
ratings=[]
country=[]
valid=True
time.sleep(5)
try:
    ads = driver.find_elements('class name', "info--ifj7U")
except:
    valid=False

if valid==True:
    for page in range(2):

        for i in range(len(ads)):

            ad = ads[i]
            try:
                reviews.append(ad.find_element('class name', 'rating__review--ygkUy').text)
            except:
                reviews.append('NONE')

            try:
                m=f'//*[@id="root"]/div/div[2]/div/div/div[1]/div[2]/div[{i+1}]/div/div/div[2]/div[1]/i'
                mall = ad.find_element('xpath', m)
                dmall.append('DMALL')
            except:
                dmall.append('Other')

            img=f'//*[@id="root"]/div/div[2]/div/div/div[1]/div[2]/div[{i+1}]/div/div/div[1]/div[1]/a/img'
            link = ad.find_element('xpath', img)
            names.append(ad.find_element('class name', 'title--wFj93').text)
            prices.append(ad.find_element('class name', 'price--NVB62').text)
            link.click()
            time.sleep(7)

            try:
                if 'Daraz' in driver.find_element('xpath', '//*[@id="module_seller_delivery"]/div/div/div[2]/div[2]/div/div/div/div[1]/div').text:
                    country.append('Pakistan')
                else:
                    country.append('Overseas')
            except:
                country.append('Pakistan')

            review = driver.find_element('id', 'module_product_review')
            driver.execute_script("arguments[0].scrollIntoView(true)", review)
            time.sleep(5)

            try:
                ratings.append(driver.find_element('xpath', '//*[@id="module_product_review"]/div/div/div[1]/div[2]/div/div/div[1]/div[1]/span[1]').text)
            except:
                ratings.append('NONE')

            driver.back()
            time.sleep(5)
            ads = driver.find_elements('class name', "info--ifj7U")

        if(len(names)<=80):
            button=driver.find_element('xpath', '//*[@id="root"]/div/div[2]/div/div/div[1]/div[3]/div/ul/li[3]')
            if button.is_enabled():
                button.click()

else:
    print('Error no data')

driver.quit()


In [ ]:
data={'Name':names,'Price':prices,'Stars':ratings,'Reviews':reviews,'Daraz Mall':dmall,'Country':country}
df=pd.DataFrame(data)
df

,Name,Price,Stars,Reviews,Daraz Mall,Country
0,Hontinga for Realme C21Y C25Y Case Luxury 6D P...,Rs. 443,4.7,(97),DMALL,Overseas
1,Hontinga for Vivo S1 Case Luxury 6D Plating So...,Rs. 506,4.8,(121),DMALL,Pakistan
2,Hontinga for Redmi Note 11 / Note 11S Back Cov...,Rs. 443,4.6,(101),DMALL,Overseas
3,Underwater Waterproof Rainproof Mobile Case PV...,Rs. 109,4.8,(328),Other,Pakistan
4,"Google Pixel 4 XL , Full Back Protection Sheet...",Rs. 92,4.4,(47),Other,Pakistan
...,...,...,...,...,...,...
75,"Ask for Any Model, CyberPunk 2077 Design Full ...",Rs. 145,4.7,(78),Other,Pakistan
76,itel Vision 1 Pro Camera Protection ShockProof...,Rs. 121,4.6,(42),Other,Pakistan
77,"Oppo F21 Pro 4G / Reno 7 4G , Full Back Protec...",Rs. 92,4.4,(68),Other,Pakistan
78,Hontinga for Redmi Note 11 / Note 11S Back Cov...,Rs. 435,4.8,(82),DMALL,Overseas


# Question 3

<!-- Which Products are on Flash sale? Display Product name, Actual Price, Discounted Price, Discount Percentage, Rating, No. of Reviews, Top 3 reviews displayed.

Display output must be like this. Products may vary. -->

If a user wants to make a store to sell something, lets say "Mobile Covers". He/She wants to see folowing things:
1. Top 10 Sellers store names
2. Daraz Mall Store/ Private store
3. Total "Mobile Covers" listings on his store
4. Avg Prices
5. Avg Units selling
6. Positive Seller Ratings
7. Seller Country

In [ ]:
q = input("Please enter search term: ").replace(' ', '+')
url = f'https://www.daraz.pk/catalog/?q={q}'
options = Options()
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
driver.get(url)
driver.maximize_window()
names=[]
dmall=[]
ratings=[]
country=[]
total=[]
followers=[]
avg_price=[]
valid=True
time.sleep(5)
try:
    ads = driver.find_elements('class name', "info--ifj7U")
except:
    valid=False

if valid==True:
    for i in range(10):
        prices=[]
        driver.find_element('xpath','//*[@id="root"]/div/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div[2]').click()
        time.sleep(5)
        brand=f'//*[@id="root"]/div/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div[1]/label[{i+1}]/span[2]'
        ele=driver.find_element('xpath',brand)
        names.append(ele.text)
        time.sleep(5)
        ele.click()
        time.sleep(5)

        try:
            total.append(driver.find_element('xpath','//*[@id="root"]/div/div[2]/div/div/div[1]/div[1]/div/div[1]/div/div/span[1]').text.split(' ')[0])
        except:
            total.append('NONE')
        ad=driver.find_elements('class name', "info--ifj7U")
        for k in range(1):
            ads=ad[k]
            try:
                mall = ads.find_element('xpath', '//*[@id="root"]/div/div[2]/div/div/div[1]/div[3]/div[1]/div/div/div[2]/div[1]/i')
                dmall.append('DMALL')
            except:
                dmall.append('Private')

        time.sleep(5)
        p=driver.find_elements('class name', "info--ifj7U")
        time.sleep(5)
        for l in range(0,len(p)):
            pri=p[l]
            time.sleep(3)
            prices.append(pri.find_element('class name', 'price--NVB62').text.split(' ')[1])

        su=0
        for pri in prices:
            if ','in pri:
                pri=pri.replace(',','')
            su=su+int(pri)
        length=len(prices)
        avg=su/length
        avg_price.append(avg)
        try:
            link = driver.find_element('xpath', '//*[@id="root"]/div/div[2]/div/div/div[1]/div[3]/div[1]/div/div/div[1]/div[1]/a/img')
        except:
            break
        link.click()
        time.sleep(5)

        try:
                if 'Daraz' in driver.find_element('xpath', '//*[@id="module_seller_delivery"]/div/div/div[2]/div[2]/div/div/div/div[1]/div').text:
                    country.append('Pakistan')
                else:
                    country.append('Overseas')
        except:
                country.append('Pakistan')

        time.sleep(5)
        try:
            ratings.append(driver.find_element('xpath','//*[@id="module_seller_info"]/div/div[2]/div[1]/div[2]').text)
        except:
            rating.append('None')

        time.sleep(5)
        try:
            driver.find_element('xpath','/html/body/div[6]/div/div[2]/div/span/i').click()
        except:
            pass
        time.sleep(5)
        shop=driver.find_element('xpath','//*[@id="module_seller_info"]/div/div[3]/a')
        time.sleep(5)
        shop.click()
        time.sleep(5)

        try:
            follow=driver.find_element('xpath','//*[@id="pi-component-container"]/div/div[2]/div/div/div/div[2]/div[1]/div/div/div/div[2]/div[2]/div').text
            followers.append(follow)
        except:
            followers.append('None')

        time.sleep(3)

        driver.back()
        time.sleep(7)
        driver.back()
        time.sleep(6)
        driver.find_element('xpath',f'//*[@id="root"]/div/div[2]/div/div/div[2]/div/div[2]/div[2]/div/div[1]/label[{i+1}]/span[1]').click()
        time.sleep(5)

driver.quit()

Please enter search term: mobile covers


C:\Users\FB TRADERS\AppData\Local\Temp\ipykernel_6152\2266569176.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [ ]:
for i in followers:
    i=i.replace('Followers','')
data={'Store Name':names,'Daraz Mall':dmall,'Country':country,'Average_Price':avg_price,'Listing':total,'Seller Rating':ratings,'Followers':followers}
df=pd.DataFrame(data)
df

,Store Name,Daraz Mall,Country,Average_Price,Listing,Seller Rating,Followers
0,Hontinga,DMALL,Overseas,472.125000,195481,93%,30609 Followers
1,Jizetin,DMALL,Overseas,487.775000,57860,94%,4261 Followers
2,UCUC,DMALL,Overseas,456.400000,64517,93%,7912 Followers
3,FELMYST,DMALL,Overseas,761.475000,55036,92%,1824 Followers
4,LENUO,DMALL,Overseas,556.250000,4714,91%,14893 Followers
5,Lereach,DMALL,Overseas,712.500000,2976,89%,2665 Followers
6,Biruiqu,Private,Overseas,930.666667,6,88%,5911 Followers
7,SGP Hybrid,DMALL,Overseas,607.100000,270,86%,546 Followers
8,Leweisi,Private,Overseas,583.750000,150,94%,381 Followers
9,Nillkin,DMALL,Overseas,1366.500000,259,92%,13151 Followers
